In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
import tensorflowjs as tfjs

import wandb
from wandb.keras import WandbCallback

import dataset
import schedulers

assert tf.config.list_physical_devices('GPU')

In [3]:
LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(units=500, maxlen=64):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model


In [4]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 

def get_xy(d):
    if d is None:
        return None
    x = d.normalized
    y = {'N': d.niqqud, 'D': d.dagesh, 'S': d.sin }
    return (x, y)


In [5]:
BATCH_SIZE = 64
MAXLEN = 64

In [26]:
np.random.seed(1)

data = {}
data['mix'] = dataset.load_data([
    'hebrew_diacritized_private/poetry',
    'hebrew_diacritized_private/rabanit',
    'hebrew_diacritized_private/pre_modern'], validation_rate=0, maxlen=MAXLEN)

data['modern'] = dataset.load_data([
    'hebrew_diacritized/modern'], validation_rate=0, maxlen=MAXLEN)

In [32]:
%env WANDB_MODE dryrun
maxlen = MAXLEN

def experiment(architecture):
    UNITS=500
    np.random.seed(2)
    model = architecture(units=UNITS)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

    model.save_weights('./checkpoints/uninit')
    
    
    config = {
        'batch_size': BATCH_SIZE,
        'maxlen': MAXLEN,
        'batch_size': BATCH_SIZE,
        'units': UNITS,
        'model': model,
        'order': [
            ('mix',    [(30e-4, 80e-4, 1e-4)], 'mix'),
            ('modern', [(50e-4, 50e-4, 1e-5)], 'modern'),
            ('modern', [(50e-4, 50e-4, 1e-5),
                        #(50e-4, 50e-4, 1e-5),
                       ], 'modern_over'),
        ],
    }

    run = wandb.init(project="dotter",
                     group="architecture",
                     name=f'model_{architecture.__name__}',
                     tags=['CLR', 'ordered'],
                     config=config)

    with run:
        for kind, clrs, save in config['order']:
            train, validation = data[kind]

            training_data = (x, y) = get_xy(train)
            validation_data = get_xy(validation)

            wandb_callback = WandbCallback(log_batch_frequency=10, training_data=training_data, validation_data=validation_data,
                                           log_weights=True)

            for clr in clrs:
                scheduler = schedulers.CircularLearningRate(*clr)
                scheduler.set_dataset(train, BATCH_SIZE)
                callbacks = [wandb_callback, scheduler]
                history = model.fit(x, y, validation_data=validation_data, batch_size=BATCH_SIZE, verbose=1, callbacks=callbacks)
            model.save(os.path.join(wandb.run.dir, save + ".h5"))
            model.save_weights('./checkpoints/' + save)
            return model

def full(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_first_add(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Dense(units, activation='relu')(layer)

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def share_weights(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    lstm = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')
    layer = lstm(embed)
    layer = layers.add([layer, lstm(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_batchnorm(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_last_add(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.Dense(units, activation='relu')(layer)

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_last_dense(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_batchnorm(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def linear_last_dense(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(units)(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def linear_last_dense_no_batchnorm(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.add([embed, layers.Dense(units)(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def linear_last_dense_no_batchnorm_share_weights(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    lstm = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')
    layer = lstm(embed)
    layer = layers.add([layer, lstm(layer)])
    layer = layers.add([embed, layers.Dense(units)(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def linear_last_dense_no_batchnorm_no_first_add(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([embed, layers.Dense(units)(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def linear_last_dense_no_batchnorm_no_last_add(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.Dense(units)(layer)
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def no_batchnorm_no_last_dense(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

def simple_bilstm(units):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    return keras.Model(inputs=inp, outputs=outputs)

for architecture in [linear_last_dense_no_batchnorm]:
    model = experiment(architecture)

env: WANDB_MODE=dryrun


2410/2410 [==============================] - 246s 102ms/step - loss: 0.5250 - N_loss: 0.2957 - D_loss: 0.1168 - S_loss: 0.1126 - N_accuracy: 0.8963 - D_accuracy: 0.9525 - S_accuracy: 0.9689 3:49 - loss: 1.9619 - N_loss: 1.2121 - D_loss: 0.4219 - S_loss: 0.3279 - N_accuracy: 0.5639 - D_accuracy: 0.8065 - - ETA: 3:47 - loss: 1.8416 - N_l - ETA: 3:38 - loss: 1.4407 - N_loss: 0.8800 - D_loss: 0.3063 - S_loss: 0.2544 - N_accuracy: 0.6810 - D_accuracy: 0.8 - ETA: 3:36 - loss: 1.3827 - N_loss: 0.8424 - D_loss: 0.2946 - S_loss: 0.2458 - N_accuracy: 0.6947 - D_accuracy - ETA: 3:33 - loss: 1.3209 - N_loss: 0.8032 - D_loss: 0.2819 - S_loss: 0.2358 - N_accuracy: 0.7092 - D_accuracy: 0.8748 - S_accuracy: 0 - ETA: 2:52  - ETA: 2:39 - loss: 0.8213 - N_loss: 0.4826 - D_loss: 0.1789 - S_loss: 0.1598 - N_accuracy: 0.8275 - D_accuracy: 0.9243 - S_ - ETA: 2:38 - loss: 0.8147 - N_loss: 0.4785 - D_loss: 0.1775 - S_lo - ETA: 2:30 - loss: 0.7877 - N_loss: 0.4610 - D_loss: 0.1720 - S_loss: 0.1547 - N_accuracy:

In [ ]:
model.load_weights('./checkpoints/modern_over')

model.compile()
model.save('modern.h5')
tfjs.converters.save_keras_model(model, '.')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
np.random.seed(3)
test, _ = dataset.load_data(['test/modernTestCorpus/'], 0, MAXLEN)

In [30]:
# model = linear_last_dense_no_batchnorm(units=500)
# model.load_weights('./checkpoints/modern_over')
x = test.normalized
y = {'N': test.niqqud, 'D': test.dagesh, 'S': test.sin }

model.compile(loss=sparse_categorical_crossentropy,
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

_ = model.evaluate(x=x, y=y, batch_size=BATCH_SIZE)

10/10 [==============================] - 0s 39ms/step - loss: 0.4038 - N_loss: 0.2181 - D_loss: 0.0896 - S_loss: 0.0961 - N_accuracy: 0.9291 - D_accuracy: 0.9650 - S_accuracy: 0.9722


In [31]:
model.load_weights('./checkpoints/modern_over')

def real_evaluation(data, s=slice(0, None), print_comparison=True):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], batch, actual_niqqud, actual_dagesh, actual_sin)
    expected = dataset.merge(data.text[s], batch, expected_niqqud, expected_dagesh, expected_sin)
    total_letters = []
    total_words = []
    for i, (b, a, e) in enumerate(zip(batch, actual, expected)):
        letters = []
        letters.extend(expected_niqqud[i][expected_niqqud[i]>0] == actual_niqqud[i][expected_niqqud[i]>0])
        letters.extend(expected_dagesh[i][expected_dagesh[i]>0] == actual_dagesh[i][expected_dagesh[i]>0])
        letters.extend(expected_sin[i][expected_sin[i]>0] == actual_sin[i][expected_sin[i]>0])
        total_letters.extend(letters)
        words = []
        for aw, ew in zip(a.split(), e.split()):
            if len([x for x in 'אבגדהוזחטיכלמנסעפצקרשתךםןףץ' if x in aw]) > 1:
                words.append(aw == ew)
                if print_comparison and aw != ew:
                    print(aw, ew)
        total_words.extend(words)
        if print_comparison:
            print('מצוי: ', a)
            print('רצוי: ', e)
            print(f'{np.mean(letters):.2%} ({len(letters)-np.sum(letters)} out of {len(letters)})')
            print(f'{np.mean(words):.2%} ({len(words)-np.sum(words)} out of {len(words)})')
            print()
    print(f'letters: {np.mean(total_letters):.2%}, words: {np.mean(total_words):.2%}')

real_evaluation(test, s=slice(0, None), print_comparison=False)  #  letters: 94.88%, words: 77.43%

letters: 94.88%, words: 77.43%


In [ ]:
import hebrew
import dataset

'len'